## Basic Pandas Examples

This notebook will walk you through some very basic Pandas concepts. We will start with importing typical data science libraries:

In [1]:
import pandas as pd
import numpy as np

### Series

Series is like a list or 1D-array, but with index. All operations are index-aligned.

In [2]:
a = pd.Series(range(1,10))
print(a)

0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
8    9
dtype: int64


In [3]:
b = pd.Series(["I","like","to","use","Python","and","Pandas","very","much"],index=range(0,9))
print(b)

0         I
1      like
2        to
3       use
4    Python
5       and
6    Pandas
7      very
8      much
dtype: object


One of the frequent usage of series is **time series**. In time series, index has a special structure - typically a range of dates or datetimes. We can create such an index with `pd.date_range`.

Suppose we have a series that shows the amount of product bought every day, and we know that every sunday we also need to take one item for ourselves. Here is how to model that using series:

## DataFrame

A dataframe is essentially a collection of series with the same index. We can combine several series together into a dataframe. Given `a` and `b` series defined above:

We can also use Series as columns, and specify column names using dictionary:

In [8]:
df = pd.DataFrame({ 'A' : a, 'B' : b })
df

,A,B
0,1,I
1,2,like
2,3,to
3,4,use
4,5,Python
5,6,and
6,7,Pandas
7,8,very
8,9,much


#### Retrieving DataFrame Information

In [13]:
df.head()
# df.tail()

,A,B
0,1,I
1,2,like
2,3,to
3,4,use
4,5,Python


In [16]:
df.shape

(9, 2)

In [18]:
df.columns

Index(['A', 'B'], dtype='object')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       9 non-null      int64 
 1   B       9 non-null      object
dtypes: int64(1), object(1)
memory usage: 272.0+ bytes


In [20]:
df.describe()

,A
count,9.000000
mean,5.000000
std,2.738613
min,1.000000
25%,3.000000
50%,5.000000
75%,7.000000
max,9.000000


**Selecting columns** from DataFrame can be done like this:

In [9]:
print(f"Column A (series):\n{df['A']}")

Column A (series):
0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
8    9
Name: A, dtype: int64


In [10]:
print(f"Columns B and A (DataFrame):\n{df[['B','A']]}")

Columns B and A (DataFrame):
        B  A
0       I  1
1    like  2
2      to  3
3     use  4
4  Python  5
5     and  6
6  Pandas  7
7    very  8
8    much  9


**Selecting rows** based on filter expression:

In [11]:
df[df['A']<5]

,A,B
0,1,I
1,2,like
2,3,to
3,4,use


The way it works is that expression `df['A']<5` returns a boolean series, which indicates whether expression is `True` or `False` for each elemens of the series. When series is used as an index, it returns subset of rows in the DataFrame. Thus it is not possible to use arbitrary Python boolean expression, for example, writing `df[df['A']>5 and df['A']<7]` would be wrong. Instead, you should use special `&` operation on boolean series:

In [12]:
df[(df['A']>5) & (df['A']<7)]

,A,B
5,6,and


**Creating new computable columns**. We can easily create new computable columns for our DataFrame by using intuitive expressions. The code below calculates divergence of A from its mean value.

In [22]:
df['DivA'] = df['A']-df['A'].mean()
df

,A,B,DivA
0,1,I,-4.0
1,2,like,-3.0
2,3,to,-2.0
3,4,use,-1.0
4,5,Python,0.0
5,6,and,1.0
6,7,Pandas,2.0
7,8,very,3.0
8,9,much,4.0


What actually happens is we are computing a series, and then assigning this series to the left-hand-side, creating another column.

In [23]:
# WRONG: df['ADescr'] = "Low" if df['A'] < 5 else "Hi"
df['LenB'] = len(df['B']) # Wrong result

In [24]:
df['LenB'] = df['B'].apply(lambda x: len(x))
# or
df['LenB'] = df['B'].apply(len)
df

,A,B,DivA,LenB
0,1,I,-4.0,1
1,2,like,-3.0,4
2,3,to,-2.0,2
3,4,use,-1.0,3
4,5,Python,0.0,6
5,6,and,1.0,3
6,7,Pandas,2.0,6
7,8,very,3.0,4
8,9,much,4.0,4


**Selecting rows based on numbers** can be done using `iloc` construct. For example, to select first 5 rows from the DataFrame:

In [32]:
df.iloc[:5, :]
# df.iloc[:5]

,A,B,DivA,LenB
0,1,I,-4.0,1
1,2,like,-3.0,4
2,3,to,-2.0,2
3,4,use,-1.0,3
4,5,Python,0.0,6


#### Selecting col based on numbers

In [33]:
df.iloc[:, :2]

,A,B
0,1,I
1,2,like
2,3,to
3,4,use
4,5,Python
5,6,and
6,7,Pandas
7,8,very
8,9,much


**Grouping** is often used to get a result similar to *pivot tables* in Excel. Suppose that we want to compute mean value of column `A` for each given number of `LenB`. Then we can group our DataFrame by `LenB`, and call `mean`:

In [53]:
df.groupby(by='LenB').mean()

,A,DivA
LenB,,
1,1.000000,-4.000000
2,3.000000,-2.000000
3,5.000000,0.000000
4,6.333333,1.333333
6,6.000000,1.000000


If we need to compute mean and the number of elements in the group, then we can use more complex `aggregate` function:

In [58]:
df.groupby(by='LenB') \
 .aggregate({ 'DivA' : len, 'A' : lambda x: x.mean() }) \
 .rename(columns={ 'DivA' : 'Count', 'A' : 'Mean'})

,Count,Mean
LenB,,
1,1,1.000000
2,1,3.000000
3,2,5.000000
4,3,6.333333
6,2,6.000000



This overview covers most important concepts of Pandas, however, the library is very rich, and there is no limit to what you can do with it! Let's now apply this knowledge for solving specific problem.